# Conversation Chat Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain/blob/main/examples/conversation_chat_model.ipynb)

In [1]:
%%capture
!pip install langchain
!pip install openai

In [19]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationEntityMemory

from langchain.chains import ConversationChain
from langchain.agents import initialize_agent
from langchain.agents import load_tools

from langchain.utilities import SerpAPIWrapper

In [ ]:
import os

os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""

### Memoryの種類
* https://langchain.readthedocs.io/en/latest/modules/memory/how_to_guides.html
* ConversationBufferMemory
    * 会話記録を保持し，プロンプトに過去会話として入れ込む
* ConversationSummaryMemory
    * 会話の要約を保存する
* ConversationEntityMemory
    * 会話中の固有名詞などを抽出して保存する  

### ConversationChainを使った実装

In [24]:
llm = OpenAI(temperature=0)

memory = ConversationBufferMemory()


agent = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory
)

agent.predict(input="はじめまして")
agent.predict(input="あなたはどんなお仕事をされていますか?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: はじめまして
AI:



> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: はじめまして
AI:  はじめまして！私はAIです。あなたの名前は何ですか？
Human: あなたはどんなお仕事をされていますか?
AI:



> Finished chain.


' 私はあなたのデジタルアシスタントです。私はあなたのスケジュールを管理したり、あなたの質問に答えたり、あなたのデータを保存したりするのを手伝います。'

In [6]:
# メモリに保存されているデータの確認
memory.load_memory_variables({})

{'history': 'Human: はじめまして\nAI:  はじめまして！私はAIです。あなたの名前は何ですか？\nHuman: あなたはどんなお仕事をされていますか?\nAI:  私はあなたのデジタルアシスタントです。私はあなたのスケジュールを管理したり、あなたの質問に答えたり、あなたのデータを保存したりするのを手伝います。'}

### 「chat-conversational-react-description」agentを使った実装

In [22]:
tools = load_tools(["serpapi"], llm=OpenAI(temperature=0))


# モデルの読み込み
llm=ChatOpenAI(temperature=0)

# 記憶の追加
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    llm = llm,
    tools = tools, 
    agent="chat-conversational-react-description", # 会話用のエージェント
    verbose=True, 
    memory=memory
    )

agent.run(input="株式会社ブレインパッドについて教えてください")



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "株式会社ブレインパッドとは"
}
Observation: ブレインパッドは、2004年の創業以来 変革を目指す企業と共に最前線を走り続ける、データ活用推進パートナーのパイオニアです。
Thought:

{
    "action": "Final Answer",
    "action_input": "株式会社ブレインパッドは、データ活用推進パートナーのパイオニアであり、2004年の創業以来、変革を目指す企業と共に最前線を走り続けています。"
}

> Finished chain.


'株式会社ブレインパッドは、データ活用推進パートナーのパイオニアであり、2004年の創業以来、変革を目指す企業と共に最前線を走り続けています。'